## Mở đầu

**Chủ đề:** Nén và phát hiện chuỗi bất thường bằng Trie và phân tích thống kê

**Sinh viên:** Trần Tấn Phát, Nguyễn Anh Quân

***Colab notebook này phục vụ cho việc demo chương trình.***

## Thiết lập

**[Cần chỉnh sửa lệnh git clone khi merge về master]**

In [1]:
# Clone repo
!git clone --branch readme-colab https://github.com/quannguyen2412899/SCAD-TSA.git
%cd SCAD-TSA

Cloning into 'SCAD-TSA'...
remote: Enumerating objects: 352, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 352 (delta 15), reused 26 (delta 10), pack-reused 301 (from 1)
Receiving objects: 100% (352/352), 3.32 MiB | 5.03 MiB/s, done.
Resolving deltas: 100% (158/158), done.
/content/SCAD-TSA


In [2]:
# Cài đặt Graphviz và python pandas
!apt-get install graphviz
%pip install graphviz pandas

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


## Biên dịch

In [3]:
!mkdir -p bin
# Biên dịch module preprocess
!g++ -std=c++17 -I./include src/preprocess.cpp src/Preprocessor.cpp -o bin/preprocess
# Biên dịch module analyze
!g++ -std=c++17 -I./include src/analyze.cpp src/Analysis.cpp src/StatTrie.cpp -o bin/analyze
# Biên dịch module visualize
!g++ -std=c++17 -I./include src/visualize.cpp -o bin/visualize
# Biên dịch chương trình chính
!g++ -std=c++17 -I./include src/main_pipeline.cpp -o main_pipeline

## Chuẩn bị dữ liệu


In [4]:
!mkdir data results

In [5]:
import random
import string

# Cấu hình file output
output_file = "data/sample_generated.log"

print("🔄 Đang sinh dữ liệu giả lập...")

with open(output_file, "w") as f:
    # ---------------------------------------------------------
    # 1. DỮ LIỆU BÌNH THƯỜNG (NORMAL TRAFFIC)
    # - Tần suất: Cao
    # - Độ dài: Trung bình, phổ biến
    # - Entropy: Thấp
    # ---------------------------------------------------------
    normal_patterns = [
        "INFO: GET /index.html 200",
        "INFO: GET /home.php 200",
        "INFO: GET /img/logo.png 200",
        "INFO: GET /img/banner.jpg 200",
        "INFO: POST /login 200"
    ]

    # Sinh 5000 dòng bình thường
    for _ in range(5000):
        f.write(random.choice(normal_patterns) + "\n")

    # ---------------------------------------------------------
    # 2. BẤT THƯỜNG TẦN SUẤT (FREQUENCY ANOMALY)
    # - Đặc điểm: Chuỗi duy nhất, chưa từng thấy.
    # ---------------------------------------------------------
    # Kẻ tấn công thử lệnh shutdown hệ thống (chỉ 1 lần)
    f.write("CRITICAL: SYSTEM SHUTDOWN INITIATED BY REMOTE HOST\n")
    print("-> Gài 1 Bất thường Tần suất (Shutdown Command).")

    # ---------------------------------------------------------
    # 3. BẤT THƯỜNG ĐỘ DÀI (LENGTH ANOMALY - Rare Length)
    # - Đặc điểm: Độ dài mà 5000 dòng kia không có.
    # ---------------------------------------------------------
    # Tấn công Buffer Overflow: Chuỗi dài 500 ký tự 'A'
    buffer_overflow = "ERROR: Input buffer overflow " + ("A" * 500)
    f.write(buffer_overflow + "\n")
    print("-> Gài 1 Bất thường Độ dài (Buffer Overflow - 500 chars).")

    # ---------------------------------------------------------
    # 4. BẤT THƯỜNG ENTROPY CAO (ENTROPY ANOMALY)
    # - Đặc điểm: Tại một node, nhánh rẽ ra quá nhiều hướng (Hỗn loạn).
    # ---------------------------------------------------------
    # Tấn công Fuzzing: Thử 100 ký tự rác khác nhau vào tham số 'token='
    # Tiền tố chung: "WARN: Invalid token="
    # Hậu tố: Thay đổi liên tục -> Node sau dấu '=' sẽ có 100 nhánh con, mỗi nhánh count=1
    # => Entropy tại node '=' sẽ RẤT CAO.

    fuzzing_chars = string.ascii_letters + string.digits + "!@#$%^&*()"
    for i in range(100):
        # Chọn ngẫu nhiên 1 ký tự để tạo nhánh ngay lập tức
        char = fuzzing_chars[i % len(fuzzing_chars)]
        f.write(f"WARN: Invalid token={char}...\n")

    print("-> Gài 100 dòng Bất thường Entropy Cao (Fuzzing Attack).")

print(f"✅ Đã tạo file: {output_file}")

🔄 Đang sinh dữ liệu giả lập...
-> Gài 1 Bất thường Tần suất (Shutdown Command).
-> Gài 1 Bất thường Độ dài (Buffer Overflow - 500 chars).
-> Gài 100 dòng Bất thường Entropy Cao (Fuzzing Attack).
✅ Đã tạo file: data/sample_generated.log


## Chạy thử chương trình

#### Chạy pipeline

In [6]:
# Phân tích dữ liệu sample, hiển thị cây trie thu gọn, chọn phần vị tần suất, độ dài và entropy lần lượt là P1, P1, P99
!./main_pipeline data/sample_generated.log results --visual-partial --perc-freq=1 --perc-len=1 --perc-entropy=99

========== Preprocess ==========
Cleaned data exported to: results/cleaned_data.txt
========== Analyze and report ==========
Report is saved at: results/overall_report.txt
CSV is saved at: results/all_entries.csv
CSV is saved at: results/frequency_anomalies.csv
CSV is saved at: results/length_anomalies.csv
CSV is saved at: results/entropy_anomalies.csv
JSON is saved at: results/partial_trie.json
========== Visualize trie ==========
dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.477277 to fit
Visualization saved at: results/partial_trie.png

Results at: results


#### Hiển thị kết quả
##### Báo cáo tổng quan

In [7]:
!cat results/overall_report.txt

==================== TRIE ANALYSIS REPORT ====================

------------------------ Trie statistics -----------------------

- Total inserted words: 5102
- Total unique words: 53
- Total unique-word characters: 1808
- Total nodes: 865
- Compressed rate (total unique-word characters / total nodes): 2.09017

----------------------- Extremum statistics ------------------------

Word frequency:
- Max frequency: 1059
- Min frequency: 1

Word length (depth):
- Max length: 529
- Min length: 21

Entropy (local node entropy):
- Max entropy: 5.34268
- Min entropy: 0

-------------------- ANOMALIES DETECTION --------------------

-------------------------- Thresholds ----------------------------

- Word frequency threshold (1% lower percentile): 1
- Length frequency threshold (1% lower percentile): 1
- Entropy threshold (99% upper percentile): 5.34268

-------------------- Anomalies: frequency-based --------------------

critical: system shutdown initiated by remote host, frequency = 1
error

##### Danh sách bất thường

In [13]:
# Danh sách bất thường tần suất
import pandas as pd
df = pd.read_csv("results/frequency_anomalies.csv")
df.index = df.index + 1
display(df)

,String,Kind,Frequency,Length,Length frequency,Entropy,Rate,Anomaly
1,critical: system shutdown initiated by remote ...,word,1,50,1,0,0.000196,frequency/length
2,error: input buffer overflow aaaaaaaaaaaaaaaaa...,word,1,529,1,0,0.000196,frequency/length
3,warn: invalid token=!...,word,1,24,100,0,0.000196,frequency
4,warn: invalid token=#...,word,1,24,100,0,0.000196,frequency
5,warn: invalid token=$...,word,1,24,100,0,0.000196,frequency
6,warn: invalid token=%...,word,1,24,100,0,0.000196,frequency
7,warn: invalid token=&...,word,1,24,100,0,0.000196,frequency
8,warn: invalid token=(...,word,1,24,100,0,0.000196,frequency
9,warn: invalid token=)...,word,1,24,100,0,0.000196,frequency
10,warn: invalid token=*...,word,1,24,100,0,0.000196,frequency


In [14]:
# Danh sách bất thường độ dài
import pandas as pd
df = pd.read_csv("results/length_anomalies.csv")
df.index = df.index + 1
display(df)

,String,Kind,Frequency,Length,Length frequency,Entropy,Rate,Anomaly
1,critical: system shutdown initiated by remote ...,word,1,50,1,0,0.000196,frequency/length
2,error: input buffer overflow aaaaaaaaaaaaaaaaa...,word,1,529,1,0,0.000196,frequency/length


In [15]:
# Danh sách bất thường entropy
import pandas as pd
df = pd.read_csv("results/entropy_anomalies.csv")
df.index = df.index + 1
display(df)

,String,Kind,Frequency,Length,Length frequency,Entropy,Rate,Anomaly
1,warn: invalid token=,prefix,100,20,0,5.34268,0.0196,entropy


Ngoài ra còn có danh sách tất cả các chuỗi/tiền tố được thống kê trong ```results/all_entries.csv```

In [16]:
import pandas as pd
df = pd.read_csv("results/all_entries.csv")
df.index = df.index + 1
display(df)

,String,Kind,Frequency,Length,Length frequency,Entropy,Rate,Anomaly
1,warn: invalid token=,prefix,100,20,0,5.34268,0.0196,entropy


##### Hiển thị cây Trie

In [12]:
from IPython.display import Image, display
display(Image("results/partial_trie.png"))